Idee:
- bei allen Motionen, Postulaten und Interpellationen schauen, von wem sie eingereicht wurden
- dann jene rausfiltern, bei denen es Mitglieder verschiedener Parteien waren
    - die werden dann später für einen Graph in Beziehung zueinander gesetzt
- noch mitnehmen, um welches Departemenet es ging
    - das wird dann als Attribut für das Edge (die Verbindung) gesetzt

### Step 1: Scrapen der GR-Website

### 1.1 Zuerst die Motionen

Vorgehen:
1. Wir rufen alle Seiten auf und ziehen uns die dort notierten Geschäftsnummern raus.
2. Diese speichern wir in einer Liste
3. Das wiederholen wir mit allen drie Geschäftsarten.
4. So können wir dann die Geschäfter direkt ansteueren, idem wir die Nummer in der Suchmaske eingeben.

Noch besser: Auf der Seite ist sogar der Direktlink zu jedem Geschäft drin, den können wir rauszupfen und in die Liste knallen.

**Infos:**

Das hier ist der Link für alle Motionen: 
https://www.gemeinderat-zuerich.ch/geschaefte/#!geschaeftsartId=1&activePage=1
Da muss man hinten einfach die Seitennummer anpassen

Plus hier die IDs für versch. Geschäfte
Motion: geschaeftsartId=1
Postulat: geschaeftsartId=2
Interpellation: geschaeftsartId=3

In [4]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys   # erlaubt Begriff an best. Stelle einzufügen
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select  #erlaubt Pulldownmenus zu operieren
import time  # um Zeitpausen einzubauen, damit er z.Bsp. die Website laden kann
import numpy as np #brauchen wir zum Aufrunden der Seitenzahlen

# öffnen der Website
driver = webdriver.Chrome(executable_path="/usr/local/bin/chromedriver")
driver.get("https://www.gemeinderat-zuerich.ch/geschaefte")

time.sleep(3)

WebDriverException: Message: 'chromedriver' executable needs to be in PATH. Please see https://sites.google.com/a/chromium.org/chromedriver/home


Hier versuche ich ein Interface zu machen, bei dem man per Eingabe die Geschäftsart und die Jahre ändern kann:

In [98]:
while True:
    typ_geschaeft = input("Geben Sie die Nummer der gewünschten Geschäftsart ein (1 = Motion, 2 = Postulat, 3 = Interpellation): ")
    typ_geschaeft = int(typ_geschaeft)
    
    if typ_geschaeft in range (1,4):
        if typ_geschaeft == 1:
            print("Sie haben Motion gewählt.")
        elif typ_geschaeft == 2:
            print("Sie haben Postulat gewählt.")
        elif typ_geschaeft == 3:
            print("Sie haben Interpellation gewählt.")
        break
    print("Sie haben eine ungültige Zahl eingegegeben. Bitte wiederholen.")

while True:
    jahr_geschaeft = input("Geben Sie das gewünschte Jahr ein (0 = alle Jahre): ")
    jahr_geschaeft = int(jahr_geschaeft)
    
    if jahr_geschaeft in range (1970,2020):
        print("Sie haben das Jahr "+str(jahr_geschaeft)+" gewählt.")
        break
    elif jahr_geschaeft == 0:
        print("Sie haben alle verfügbaren Jahre gewählt.")
        break
    print("Sie haben eine ungültige Zahl eingegegeben. Bitte wiederholen.")

Geben Sie die Nummer der gewünschten Geschäftsart ein (1 = Motion, 2 = Postulat, 3 = Interpellation): 2
Sie haben Postulat gewählt.
Geben Sie das gewünschte Jahr ein (0 = alle Jahre): 2014
Sie haben das Jahr 2014 gewählt.


In [99]:
if jahr_geschaeft == 0:
    driver.get("https://www.gemeinderat-zuerich.ch/geschaefte/#!geschaeftsartId="+str(typ_geschaeft)+"&activePage=1")
else:
    driver.get("https://www.gemeinderat-zuerich.ch/geschaefte/#!geschaeftsartId="+str(typ_geschaeft)+"&jahr="+str(jahr_geschaeft)+"&activePage=1")

time.sleep(3)

Ansteuern der einzelnen Geschäfte (bei voller Seite 10 Stück):

In [83]:
geschaefte = driver.find_elements_by_class_name("tableDataRow")

In [50]:
for einzelgeschaeft in range(0,9): 
    geschaefte[einzelgeschaeft].click()

In [51]:
page = driver.page_source.encode('utf-8')  

In [59]:
#hier ist die Geschäftsnummer für den Filetitel:
from bs4 import BeautifulSoup
soup = BeautifulSoup(page, 'html.parser')
nummer = soup.find_all("span")[2].text

'2019/524'

In [ ]:
  with open("data/geschaeft"+nummer+".htm", "wb+") as file:
        file.write(page)
        file.close()

### Jetzt setzen wir mal alles zusammen – 3 Durchläufe mit unterschiedlichen Geschäftsarten:

In [86]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys   # erlaubt Begriff an best. Stelle einzufügen
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select  #erlaubt Pulldownmenus zu operieren
import time  # um Zeitpausen einzubauen, damit er z.Bsp. die Website laden kann

In [87]:
# öffnen der Website
driver = webdriver.Chrome(executable_path="/usr/local/bin/chromedriver")
driver.get("https://www.gemeinderat-zuerich.ch/geschaefte")
time.sleep(3)

In [177]:
#Interface für Geschäftstyp und Jahr
while True:
    typ_geschaeft = input("Geben Sie die Nummer der gewünschten Geschäftsart ein (1 = Motion, 2 = Postulat, 3 = Interpellation): ")
    typ_geschaeft = int(typ_geschaeft)
    
    if typ_geschaeft in range (1,4):
        if typ_geschaeft == 1:
            print("Sie haben Motion gewählt.")
        elif typ_geschaeft == 2:
            print("Sie haben Postulat gewählt.")
        elif typ_geschaeft == 3:
            print("Sie haben Interpellation gewählt.")
        break
    print("Sie haben eine ungültige Zahl eingegegeben. Bitte wiederholen.")

while True:
    jahr_geschaeft = input("Geben Sie das gewünschte Jahr ein (0 = alle Jahre): ")
    jahr_geschaeft = int(jahr_geschaeft)
    
    if jahr_geschaeft in range (1970,2020):
        print("Sie haben das Jahr "+str(jahr_geschaeft)+" gewählt.")
        break
    elif jahr_geschaeft == 0:
        print("Sie haben alle verfügbaren Jahre gewählt.")
        break
    print("Sie haben eine ungültige Zahl eingegegeben. Bitte wiederholen.")

if jahr_geschaeft == 0:
    url = "https://www.gemeinderat-zuerich.ch/geschaefte/#!geschaeftsartId="+str(typ_geschaeft)+"&activePage="
    driver.get(url+"1")
else:
    url = "https://www.gemeinderat-zuerich.ch/geschaefte/#!geschaeftsartId="+str(typ_geschaeft)+"&jahr="+str(jahr_geschaeft)+"&activePage="
    driver.get(url+"")

time.sleep(3)

Geben Sie die Nummer der gewünschten Geschäftsart ein (1 = Motion, 2 = Postulat, 3 = Interpellation): 3
Sie haben Interpellation gewählt.
Geben Sie das gewünschte Jahr ein (0 = alle Jahre): 1990
Sie haben das Jahr 1990 gewählt.


In [178]:
#Die Anzahl Seiten-Loops ermittle ich durch den Wert "Liste der Geschäfte" geteilt durch 10
page = driver.page_source.encode('utf-8')

soup1 = BeautifulSoup(page, 'html.parser')
span = soup1.find_all("span")
treffer = int(span[85].text)
seitenzahl = int(np.ceil(treffer/10)) # hier wird mit ceil konsequent aufgerundet
seitenzahl

0

In [179]:
url

'https://www.gemeinderat-zuerich.ch/geschaefte/#!geschaeftsartId=3&jahr=1990&activePage='

In [180]:
#Jetzt kommt der übergeordnete Seitenloop:
for seite in range(1,seitenzahl+1):
    
    driver.get(url+str(seite))
    time.sleep(3)
        
    # hier finde ich heraus, wie oft der Loop stattfinden soll
    anzahl_geschaefte = len(driver.find_elements_by_class_name("tableDataRow"))

    # nacheinander ansteuern jedes Geschäfts und speichern
    for einzelgeschaeft in range(0,anzahl_geschaefte): 
        time.sleep(1)     
        geschaefte = driver.find_elements_by_class_name("tableDataRow")
        
        geschaefte[einzelgeschaeft].click()
    
        page = driver.page_source.encode('utf-8')
    
        # hier hole ich für den Filetitel die Geschäftsnummer raus und säubere sie vom "/"
        soup = BeautifulSoup(page, 'html.parser')
        nummer = soup.find_all("span")[2].text
        nummer = nummer.replace("/","_")
    
        with open("data/geschaeft"+nummer+".htm", "wb+") as file:
            file.write(page)
            file.close()
    
        # jetzt gehe ich zurück zur Übersichtsseite und lege eine kurze Pause ein, weil sonst Click z.T. nicht geht 
        driver.back()

### 